* creates `deriv/wed_df.csv`
- make and save dataframe
- uses `NvSclass_df` and `timing_df_viewing` and `timing_df_recall`

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance

pd.options.display.max_rows = 200

### cols
- sub_num
- wed_id
- onset_view
- onset_recal
- offset_view
- offset_recal

In [2]:
""" onset and offset TR for each state of each wedding """
timing_df_view = pd.read_csv('deriv/timing_df_viewing.csv',index_col=0)
print('\n',timing_df_view.head())
timing_df_recal = pd.read_csv('deriv/timing_df_recall.csv',index_col=0)
print('\n',timing_df_recal.head())
wedclass_df = pd.read_csv('deriv/NvSclass_df.csv',index_col=0)
print('\n',wedclass_df.head())


    wed_num  vid_str  onset_TR  len_TRs  offset_TR
0        0  vid1a_q        12       17         29
1        0  vid1b_q        29        6         35
2        0   vid2_q        35       11         46
3        0   vid3_q        49       15         64
4        0   vid4_q        67       16         83

    sub_num  wed_id  onset_TR  offset_TR wed_class
0        2      34         0         27       NaN
1        2      29        27         46       NaN
2        2      17        46         63       NaN
3        2      38        63         81       NaN
4        2       6        81        124       NaN

   NorS  wed_id  wed_num  sub_num
0    S      20        0        3
1    N      23        1        3
2    N       6        2        3
3    S      17        3        3
4    S      38        4        3


### init wed_df from timing_recall_df

In [21]:
wed_df = timing_df_recal.copy()
wed_df = wed_df.rename(columns={'onset_TR':'onset_recall','offset_TR':'offset_recall'})

### include col wed_view_num 

In [59]:
""" 
use `NvSclass_df.csv` to match wedding_id and wedding_viewing_order
"""
## init col
wed_df.loc[:,'wed_view_num']=int(-99)
## loop over rows
for idx,df_row in wed_df.iterrows():  
  try:
    wed_df.loc[idx,'wed_view_num'] = wedclass_df[(
                    wedclass_df.sub_num == df_row.sub_num) & (
                    wedclass_df.wed_id == df_row.wed_id
                  )].wed_num.values[0]
  except:
    print('err sub',df_row.sub_num)
    continue


err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39


### include cols onset_view and offset_view

In [58]:
"""
from `timing_df_viewing.csv` extract onset and offset of viewing 
"""
for idx,df_row in wed_df.iterrows():
  timing_df_view_wed = timing_df_view[timing_df_view.wed_num == df_row.wed_view_num]
  onset_view = timing_df_view_wed[timing_df_view_wed.vid_str.str[:len('vid1a')]=='vid1a'].onset_TR
  offset_view = timing_df_view_wed[timing_df_view_wed.vid_str.str[:len('vid5')]=='vid5'].offset_TR
  if not len(onset_view):
    print('err sub',df_row.sub_num)
    continue
  wed_df.loc[idx,'onset_view'] = onset_view.values[0]
  wed_df.loc[idx,'offset_view'] = offset_view.values[0]

err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 2
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 11
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 37
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39
err sub 39


,sub_num,wed_id,onset_recall,offset_recall,wed_class,onset_view,offset_view,wed_view_num
0,2,34,0,27,NaN,-99,-99,-99
1,2,29,27,46,NaN,-99,-99,-99
2,2,17,46,63,NaN,-99,-99,-99
3,2,38,63,81,NaN,-99,-99,-99
4,2,6,81,124,NaN,-99,-99,-99
...,...,...,...,...,...,...,...,...
475,44,34,215,245,S,783,856,8
476,44,28,245,265,S,303,375,3
477,44,1,265,294,N,879,952,9
478,44,38,294,312,S,1074,1152,11


In [22]:
wed_df.loc[:,'onset_view']=-99
wed_df.loc[:,'offset_view']=-99